In [61]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from mlxtend.regressor import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from google.colab import files

**Loading the data**

In [62]:
files.upload()

{}

In [63]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

**Preprocessing**

In [64]:
y = np.log(train_data.SalePrice)
test_id = test_data['Id']
train_test = pd.concat((train_data.loc[:, 'MSSubClass':'SaleCondition'], test_data.loc[:, 'MSSubClass':'SaleCondition']))
train_test = train_test.drop(['Utilities'], axis=1)
dropped_columns = ['BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'PoolQC', 'Street', 'SaleCondition']
train_test.drop(dropped_columns, axis=1)
train_test['street'] = pd.get_dummies(train_test.Street, drop_first=True)
train_test['condition'] = train_test.SaleCondition.apply(lambda x: 1 if x == 'Normal' else 0)
train_data= all_data[:train_data.shape[0]].select_dtypes(include=[np.number]).interpolate().dropna()
test_data = all_data[train_data.shape[0]:].select_dtypes(include=[np.number]).interpolate().dropna()

**Modeling**

In [65]:
lr = LinearRegression(n_jobs=-1)
gb = GradientBoostingRegressor(n_estimators=40, max_depth=2)
dtr = DecisionTreeRegressor()
model = StackingRegressor(regressors=[dtr, gb], meta_regressor=lr)

model.fit(train_data, y)

StackingRegressor(meta_regressor=LinearRegression(copy_X=True,
                                                  fit_intercept=True, n_jobs=-1,
                                                  normalize=False),
                  refit=True,
                  regressors=[DecisionTreeRegressor(ccp_alpha=0.0,
                                                    criterion='mse',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=...
                            

**Predicting**

In [66]:
y_pred = model.predict(train_data)

Y_pred = model.predict(test_data)

pred = np.exp(Y_pred)

**Submission**

In [67]:
output = pd.DataFrame(test_id, columns=['Id'])
output['SalePrice'] = pred
output.to_csv("submission.csv", index=False, header=True)